In [70]:
import tkinter as tk
import pygame 
import time
import datetime
import random
from mutagen.mp3 import MP3
import pandas as pd
import serial
import numpy as np
from scipy.io.wavfile import write
import soundfile
import contextlib

Generate the stimuli .wav files

In [71]:
samplerate = 44100 
duration = 1.5
frequency = 500 

t = np.linspace(0., duration, int(samplerate * duration))
signal = 0.5 * np.sin(2. * np.pi * frequency * t)
write('stimuli0.wav', samplerate, signal.astype(np.float32))

frequency = 2000

t = np.linspace(0., duration, int(samplerate * duration))
signal = 0.5 * np.sin(2. * np.pi * frequency * t)
write('stimuli1.wav', samplerate, signal.astype(np.float32))

frequency = 4000

t = np.linspace(0., duration, int(samplerate * duration))
signal = 0.5 * np.sin(2. * np.pi * frequency * t)
write('stimuli2.wav', samplerate, signal.astype(np.float32))

In [72]:
class BiosemiTrigger(serial.Serial):
    def __init__(self, Serial_Port, initial_delay = 3):
        super().__init__(Serial_Port, baudrate = 115200)
        time.sleep(initial_delay)

    def send_trigger(self, root, duration = 8, signal_byte = 0b00000010):
        if not (0 <= signal_byte <= 255):
                raise ValueError("signal_byte must be between 0 and 255")
        self.write(bytes([signal_byte]))
        #time.sleep(0.001) #1ms pulse duration
        root.after(duration, lambda: self.write(bytes([0])))

In [73]:
def create_canvas(tk_window):
    screen_width = tk_window.winfo_screenwidth()
    screen_height = tk_window.winfo_screenheight()

    canvas = tk.Canvas(tk_window, width = screen_width, height = screen_height, bg = "black", highlightthickness = 0)
    canvas.pack(fill = "both", expand = True)
    return canvas

In [ ]:
def create_cross(canvas, tk_window): 
    global timestamp
    if timestamp: 
        if timestamp[-1][2] is None:
            timestamp[-1][2] = datetime.datetime.now()
    timestamp.append(['Resting State', datetime.datetime.now(), None])

    screen_width = tk_window.winfo_screenwidth()
    screen_height = tk_window.winfo_screenheight()
    
    cross_length = 250
    line_thickness = 13
    x_center = screen_width // 2
    y_center = screen_height // 2

    horizontal_line = canvas.create_line(x_center - cross_length // 2, y_center,
                    x_center + cross_length // 2, y_center,
                    fill = "white", width = line_thickness)

    vertical_line = canvas.create_line(x_center, y_center - cross_length // 2,
                    x_center, y_center + cross_length // 2,
                    fill = "white", width = line_thickness)
    

In [76]:
def stimuli_duration(selector):
    filename = f"stimuli{selector}.wav"
    with contextlib.closing(soundfile.SoundFile(filename)) as f: #closes the .wav file after the function finishes
        frames = f.frames
        rate = f.samplerate
        duration = frames / float(rate)
        duration = int(duration * 1000)
    return duration

In [77]:
def stimuli(tk_window, selector, biosemi_trigger):
    filename = f"stimuli{selector}.wav"
    stim = pygame.mixer.Sound(filename)
    
    duration = stimuli_duration(selector)
    biosemi_trigger.send_trigger(tk_window, duration)
    stim.play()

In [ ]:
def stim_bg(canvas, tk_window, selector, biosemi_trigger):
    global timestamp
    if timestamp: 
        if timestamp[-1][2] is None:
            timestamp[-1][2] = datetime.datetime.now()
    timestamp.append(['Resting State', datetime.datetime.now(), None])
    
    canvas.delete('all')
    stimuli(tk_window, selector, biosemi_trigger)

In [92]:
def random_time():
    randomtime = int(round(random.uniform(9.0, 12.0), 1) * 1000)
    return randomtime

In [ ]:
def stimuli_cicle(time, tk_window, canvas, selector, biosemi_trigger):
    duration = stimuli_duration(selector)
    wait = random_time()

    tk_window.after(time+wait, lambda: stim_bg(canvas, tk_window, selector, biosemi_trigger))
    tk_window.after(time+wait+duration, lambda: create_cross(canvas, tk_window))
    return wait

In [ ]:
def create_csv(input):
    global timestamp
    if timestamp: 
        if timestamp[-1][2] is None:
            timestamp[-1][2] = datetime.datetime.now()
    df = pd.DataFrame(timestamp, columns = ['State', 'Begin', 'End'])
    df.drop(columns=['Unnamed: 0'])
    df.to_csv(f"{input}.csv")

In [ ]:
def start_exp(input, biosemi_trigger): 
    global timestamp
    timestamp = []

    w2 = tk.Tk()
    w2.title("Experiment")
    w2.attributes("-fullscreen", True)

    canvas = create_canvas(w2)
    create_cross(canvas, w2, input)
    
    time = 0
    for i in range(3):
        wait = stimuli_cicle(time, w2, canvas, input, i, biosemi_trigger)
        time = wait + time
        
    wait_close = time + stimuli_duration(2) + 2000
    w2.after(wait_close, lambda: create_csv(input))
    w2.after(wait_close, lambda: w2.destroy())

In [82]:
biosemi_trigger = BiosemiTrigger("COM4", initial_delay = 1)

w1 = tk.Tk()
w1.geometry("300x200")
w1.title("Start")
pygame.mixer.init()


frame = tk.Frame(w1)
frame.pack(expand = True)

entry = tk.Entry(frame)
entry.pack(pady = (0, 5))

b = tk.Button(frame, text = "Start", command = lambda: start_exp(entry.get(), biosemi_trigger))
b.pack()

w1.mainloop()
biosemi_trigger.close()
pygame.quit()

In [83]:
df=pd.read_csv("test.csv")
print(df)

           State                       Begin                         End
0  Resting State  2025-05-06 15:39:01.882328  2025-05-06 15:39:07.596234
1        Stimuli  2025-05-06 15:39:07.606762  2025-05-06 15:39:09.096140
2  Resting State  2025-05-06 15:39:09.104412  2025-05-06 15:39:14.202137
3        Stimuli  2025-05-06 15:39:14.210639  2025-05-06 15:39:15.701351
4  Resting State  2025-05-06 15:39:15.710244  2025-05-06 15:39:21.706648
5        Stimuli  2025-05-06 15:39:21.714129  2025-05-06 15:39:23.206590
6  Resting State  2025-05-06 15:39:23.214881  2025-05-06 15:39:25.210525


In [86]:
ar_of_items = []

timestamp_begin = 'timestamp that begins' 
timestamp_end = 'datetime.datetime.now())'

ar_of_items.append(['Stimuli',timestamp_begin,timestamp_end])


### Sto save
df = pd.DataFrame(ar_of_items,columns=['State','Begin','End'])

print(df)

     State                  Begin                       End
0  Stimuli  timestamp that begins  datetime.datetime.now())
